In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import os
import glob

In [2]:
dataset_id = ''

In [3]:
!pwd

/home/eduardo/Desktop/Link to landmark-based-dynamic-projections/plots


In [4]:
dataset_id = 'quickdraw'
os.chdir('..')

In [5]:
projection_paths = pd.read_csv('metrics/results/{}.csv'.format(dataset_id), index_col=0).index

In [6]:
# dataset_path = '../datasets/' + dataset_id + '/'
# print(projection_paths, dataset_id)

# projection_paths =
#     ['/home/eduardo/Desktop/Link to dynamic-projections/Output/quickdraw-AE_784f_500f_500f_2000f_2f_20ep.csv',
# #        '/home/eduardo/Desktop/Link to dynamic-projections/Output/quickdraw-C2AE_32c_32c_32c_1568f_2f_2ep.csv',
#        '/home/eduardo/Desktop/Link to dynamic-projections/Output/quickdraw-VAE_784f_2048f_1024f_512f_2f_0-25drop_10ep.csv',
#        '/home/eduardo/Desktop/Link to dynamic-projections/Output/quickdraw-C2VAE_32c_64c_128c_6272f_2f_10ep.csv',
#        '/home/eduardo/Desktop/Link to dynamic-projections/Output/quickdraw-tsne_s1_30p.csv', 'Output/quickdraw-tsne_s4_30p.csv',
#        'Output/quickdraw-dtsne_200p_0-1l.csv',
#        'Output/quickdraw-umap_s1_15p.csv', 'Output/quickdraw-umap_s4_15p.csv',
#        'Output/quickdraw-pca_s1.csv', 'Output/quickdraw-pca_s4.csv',
#        'output/quickdraw-ldtsne-p30-l0_1000-ge2-le1-krandom-n-TSNE-1585997292.csv',
#        'output/quickdraw-pcadtsne-p30-l0_001000-le1-ls1_00.csv']

In [7]:
ppppp = []
for i, p in enumerate(projection_paths):
    if len(glob.glob('./output/' + p)) > 0:
        ppppp.append('./output/' + p)
    elif len(glob.glob('/home/eduardo/Desktop/Link to dynamic-projections/Output/'+p)) > 0:
        ppppp.append('/home/eduardo/Desktop/Link to dynamic-projections/Output/' + p)
projection_paths = ppppp
projection_paths

['/home/eduardo/Desktop/Link to dynamic-projections/Output/quickdraw-AE_784f_500f_500f_2000f_2f_20ep.csv',
 '/home/eduardo/Desktop/Link to dynamic-projections/Output/quickdraw-C2AE_32c_32c_32c_1568f_2f_2ep.csv',
 '/home/eduardo/Desktop/Link to dynamic-projections/Output/quickdraw-VAE_784f_2048f_1024f_512f_2f_0-25drop_10ep.csv',
 '/home/eduardo/Desktop/Link to dynamic-projections/Output/quickdraw-C2VAE_32c_64c_128c_6272f_2f_10ep.csv',
 '/home/eduardo/Desktop/Link to dynamic-projections/Output/quickdraw-tsne_s1_30p.csv',
 '/home/eduardo/Desktop/Link to dynamic-projections/Output/quickdraw-tsne_s4_30p.csv',
 './output/quickdraw-ctsne-p30.csv',
 '/home/eduardo/Desktop/Link to dynamic-projections/Output/quickdraw-dtsne_200p_0-1l.csv',
 '/home/eduardo/Desktop/Link to dynamic-projections/Output/quickdraw-umap_s1_15p.csv',
 '/home/eduardo/Desktop/Link to dynamic-projections/Output/quickdraw-umap_s4_15p.csv',
 './output/quickdraw-cumap.csv',
 '/home/eduardo/Desktop/Link to dynamic-projections/O

In [8]:
names = pd.DataFrame(projection_paths)[0].str.extract(r'.*{}-(.*).csv'.format(dataset_id), expand=False) # Clean string

In [9]:
!mkdir {'plots/anim/'+dataset_id}

mkdir: cannot create directory ‘plots/anim/quickdraw’: File exists


In [10]:
def get_method(name):
    s = name
    if s.startswith('tsne_s1'):
        return 'TF-tSNE'
    elif s.startswith('tsne_s4'):
        return 'G-tSNE'
    elif s.startswith('ctsne'):
        return 'C-tSNE'
    elif s.startswith('dtsne'):
        return 'D-tSNE'
    elif s.startswith('ldtsne'):
        return 'LD-tSNE'
    elif s.startswith('pcadtsne'):
        return 'PCD-tSNE'
    elif s.startswith('pca_s1'):
        return 'TF-PCA'        
    elif s.startswith('pca_s4'):
        return 'G-PCA'
    elif s.startswith('umap_s1'):
        return 'TF-UMAP'        
    elif s.startswith('umap_s4'):
        return 'G-UMAP'
    elif s.startswith('cumap'):
        return 'C-UMAP'
    elif s.startswith('AE'):
        return 'G-AE'
    elif s.startswith('VAE'):
        return 'G-VAE'
    else:
        return s.split('_')[0]

In [11]:
name_to_ax = {'TF-PCA' : 0,
'TF-tSNE' : 1 ,
'TF-UMAP' : 2,
'G-AE' : 5,
'G-VAE' : 6,
'G-PCA' : 7,
'G-tSNE' : 8,
'G-UMAP' : 9,
'C-tSNE' : 10,
'C-UMAP' : 11,
'D-tSNE' : 12,
'PCD-tSNE' : 15,
'LD-tSNE' : 16}


In [12]:
def get_projection_as_array(dataset_path):
    df = pd.read_csv(dataset_path, index_col=0)
    vs = df.values.reshape(len(df), -1, 2)
    return vs, df.index, vs.shape[1]

In [13]:
lims = {}
pos_interp = {}
indexes = {}
colors = {}
for p_i, p in enumerate(projection_paths):
    pos, ind, n_timesteps = get_projection_as_array(p)
    # Shuffle arrays
    perm = np.random.permutation(len(ind))
    pos = pos[perm]
    indexes[p] = ind[perm]

    # Set x and y axis limits
    x_max = max(pos[:,:,0].flatten())
    x_min = min(pos[:,:,0].flatten())
    y_max = max(pos[:,:,1].flatten())
    y_min = min(pos[:,:,1].flatten())
    x_max = x_max + (x_max - x_min) * .03
    x_min = x_min - (x_max - x_min) * .03
    y_max = y_max + (y_max - y_min) * .03
    y_min = y_min - (y_max - y_min) * .03
    lims[p] = (x_min, x_max, y_min, y_max)

    # Choose colors
    colormap = matplotlib.cm.Set3
    classes = indexes[p].str.split('-').str[0]
    unique_colors = {k:i for i,k in enumerate(np.unique(classes))}
    colors[p] = [colormap(unique_colors[c]) for c in classes] 

    # Add more points with akima interpolation
    n_nans = 9
    pos_int = []
    for points in pos:
        extended = []
        for po in points:
            extended.append(list(po))
            for i in range(n_nans):
                extended.append([np.nan, np.nan])
        df = pd.DataFrame(extended)
        df = df.interpolate(method='akima')
        df = df.dropna()
        pos_int.append(df.values)
    pos_interp[p] = np.array(pos_int)
    n_extended = len(df)

In [ ]:
anim_dir = 'plots/anim/{}'.format(dataset_id)
# Draw figures
size_trail = 5
n_proj = len(projection_paths)

for t in range(582,n_extended):
    ncols = 5
    nrows = 4
    fig, axes = plt.subplots(ncols=ncols, nrows=nrows,figsize=(ncols*5,nrows*5))   
    axes = axes.flatten()
    for i in range(ncols*nrows):
        if i not in name_to_ax.values():
            [ax.set_visible(False) for ax in axes[i:i+1]]
            
    for p_i, p in enumerate(projection_paths):
        if get_method(names[p_i]) in name_to_ax:
            # Draw trails
            ax = axes[name_to_ax[get_method(names[p_i])]]
    #         [p_i]
            ax.set_title(get_method(names[p_i]), fontsize=18)
            for i in range(len(indexes[p])):
                b_trail = max(0, t - n_nans * size_trail)
                e_trail = t + 1
                points = pos_interp[p][i,b_trail:e_trail]
                ax.plot(points[:,0], points[:,1], 
                        c=colors[p][i], lw=1, alpha=0.1);
            # Draw scatter
            ax.scatter(pos_interp[p][:,t,0], pos_interp[p][:,t,1], 
                       edgecolors='#000000', s=15, c=colors[p], alpha=0.5, zorder=4);
            # Write text with current t in the first axis
            if name_to_ax[get_method(names[p_i])] == 0:
                current_t = int(t/(n_nans+1))
                ax.text(0.01, 0.01, str(current_t), 
                        ha='left', va='bottom', fontsize=18, transform=ax.transAxes)

            ax.tick_params(axis='both', which='both', bottom=False, top=False,
                           labelbottom=False, right=False, left=False, labelleft=False)
            ax.set_xlim(lims[p][0], lims[p][1])
            ax.set_ylim(lims[p][2], lims[p][3])
            plt.tight_layout()
    #      plt.axis('scaled')

    # Save current frame
    plt.savefig('{}/proj-{}.png'.format(anim_dir, t),
                format='png', dpi=80, bbox_inches='tight')
#     plt.show()
    plt.clf()
    plt.close()
#     break

In [ ]:
# Generate gif - usually large
# n = n_extended-1
# !convert -delay 5 -loop 0 {anim_dir}/proj-%d.png[0-{n}] {anim_dir}/{dataset_id}-gif.gif

In [ ]:
os.chdir(anim_dir)
! pwd

In [ ]:
# sudo apt install mencoder

In [ ]:
# # Generate video
! ls | grep .*.png | sort -V > order.txt
! mencoder mf://@order.txt -mf fps=10:type=png -ovc x264 -x264encopts bitrate=2400:threads=2 -o {dataset_id}-avi-10.avi
# ! mencoder mf://@order.txt -mf fps=15:type=png -ovc x264 -x264encopts bitrate=2400:threads=2 -o {dataset_id}-avi-15.avi
! rm order.txt

In [ ]:
# # Remove intermediate files
# for i in range(n_extended):
#     if i % (n_nans+1) != 0:
#         r = 'proj-{}.png'.format(anim_dir, i) 
#         print('rm ' + r)
#         ! rm {r}